In [3]:
# This notebook demonstrates the architecture and parameter settings of the MentalAId model. 
# Due to stochastic initialization, training results may slightly vary. 
# To use the trained MentalAId, please refer to the model directory in the root folder.

In [1]:
import os
import sklearn
from sklearn import metrics
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.python.eager import context
from tensorflow import keras
from tensorflow.keras.layers import Input
import numpy as np
import random
import time
import math
import pandas as pd
import pylab as plt
from sklearn.model_selection import KFold
from sklearn.utils import resample

2025-05-07 10:45:06.179002: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
os.chdir("/data/home/lmx/psy_test/v1_20241016/")

In [3]:
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_USE_FRONTEND'] = '1'

os.environ["PYTHONHASHSEED"] = '0'

tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)
    
tf.config.set_soft_device_placement = False
tf.config.experimental.set_memory_growth = True
gpus = tf.config.experimental.list_physical_devices('GPU')
print("gpus:", gpus)
 

if gpus:
    tf.config.experimental.set_virtual_device_configuration(gpus[1], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), len(logical_gpus), 'Logical gpus')
    
### set background seed
seed = 42

random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.experimental.numpy.random.seed(seed)

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

x_train = np.load('./data/x_train.npy',allow_pickle=True)
y_train = np.load('./data/y_train.npy',allow_pickle=True)
print(x_train[0].shape)
x_train = np.expand_dims(x_train, -1)
x_train = x_train.astype('float32') 


num_classes = 2
input_shape = x_train.shape[1:]

gpus: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
2 2 Logical gpus
(7, 7)


2025-05-07 10:45:07.407476: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2025-05-07 10:45:07.421695: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:25:00.0 name: Tesla V100-PCIE-32GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2025-05-07 10:45:07.421951: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:5b:00.0 name: Tesla V100-PCIE-32GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2025-05-07 10:45:07.421975: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2025-05-07 10:45:07.424313: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.

In [4]:
sex_train = pd.read_csv('./data/data_clean.csv', index_col='Unnamed: 0')['Gender']

### set sex female=0 male=1 
sex_train = sex_train.replace(['F','女'],0)
sex_train = sex_train.replace(['M','男'],1)

sex_train.value_counts()

sex_train = np.array(sex_train).astype(np.float32)

age_train = pd.read_csv('./data/data_clean.csv', index_col='Unnamed: 0')['Age']

age_train = np.array(age_train).astype(np.float32)

# MentalAId with sex & gender 

## network architecture

In [8]:
# define bottleneck
class BottleNeck(tf.keras.layers.Layer):
    def __init__(self, growth_rate, drop_rate):
        super(BottleNeck, self).__init__()
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2D(filters=4 * growth_rate,
                                            kernel_size=(1, 1),
                                            strides=1,
                                            padding="same")
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filters=growth_rate,
                                            kernel_size=(3, 3),
                                            strides=1,
                                            padding="same")
        self.dropout = tf.keras.layers.Dropout(rate=drop_rate)
        
        self.listLayers = [self.bn1,
                           tf.keras.layers.Activation("relu"),
                           self.conv1,
                           self.bn2,
                           tf.keras.layers.Activation("relu"),
                           self.conv2,
                           self.dropout]

    def call(self, x):
        tf.random.set_seed(seed)
        y = x
        for layer in self.listLayers.layers:
            y = layer(y)
        y = tf.keras.layers.concatenate([x,y], axis=-1)
        return y

# define dense block
class DenseBlock(tf.keras.layers.Layer):
    def __init__(self, num_layers, growth_rate, drop_rate=0.5):
        super(DenseBlock, self).__init__()
        self.num_layers = num_layers
        self.growth_rate = growth_rate
        self.drop_rate = drop_rate
        self.listLayers = []
        for _ in range(num_layers):
            self.listLayers.append(BottleNeck(growth_rate=self.growth_rate, drop_rate=self.drop_rate))

    def call(self, x):
        tf.random.set_seed(seed)
        for layer in self.listLayers.layers:
            x = layer(x)
        return x

# define transition
class TransitionLayer(tf.keras.layers.Layer):
    def __init__(self, out_channels):
        super(TransitionLayer, self).__init__()
        self.bn = tf.keras.layers.BatchNormalization()
        self.conv = tf.keras.layers.Conv2D(filters=out_channels,
                                           kernel_size=(1, 1),
                                           strides=1,
                                           padding="same")
        self.pool = tf.keras.layers.MaxPool2D(pool_size=(2, 2),
                                              strides=2,
                                              padding="same")

    def call(self, inputs):
        tf.random.set_seed(seed)
        x = self.bn(inputs)
        x = tf.keras.activations.relu(x)
        x = self.conv(x)
        x = self.pool(x)
        return x

# define dense net
class DenseNet(tf.keras.Model):
    def __init__(self, num_init_features, growth_rate, block_layers, compression_rate, drop_rate):
        super(DenseNet, self).__init__()

        self.conv = tf.keras.layers.Conv2D(filters=num_init_features,
                                           kernel_size=(3, 3),
                                           strides=1,
                                           input_shape = (7,7,1),
                                           padding="same")
        self.bn = tf.keras.layers.BatchNormalization()
        
        self.num_channels = num_init_features
        self.dense_block_1 = DenseBlock(num_layers=block_layers[0], growth_rate=growth_rate, drop_rate=drop_rate)
        self.num_channels += growth_rate * block_layers[0]
        self.num_channels = compression_rate * self.num_channels
        self.transition_1 = TransitionLayer(out_channels=int(self.num_channels))
        self.dense_block_2 = DenseBlock(num_layers=block_layers[1], growth_rate=growth_rate, drop_rate=drop_rate)
        self.num_channels += growth_rate * block_layers[1]
        self.num_channels = compression_rate * self.num_channels
        self.transition_2 = TransitionLayer(out_channels=int(self.num_channels))
        self.dense_block_3 = DenseBlock(num_layers=block_layers[2], growth_rate=growth_rate, drop_rate=drop_rate)
        
        self.avgpool = tf.keras.layers.GlobalAveragePooling2D()
                
        # MLP layers for age and gender (this module is added)
        self.age_dense = tf.keras.layers.Dense(units=16, activation='relu', name='age_dense')
        self.gender_dense = tf.keras.layers.Dense(units=16, activation='relu', name='gender_dense')
        
        # fc
        self.fc = tf.keras.layers.Dense(units=2,
                                        activation=tf.keras.activations.softmax)

    def call(self, inputs):
        # inputs should now include image, age, gender
        image_input, age_input, gender_input = inputs
        
        # tf.random.set_seed(seed)
        x = self.conv(image_input)
        x = self.bn(x)
        x = tf.keras.activations.relu(x)

        x = self.dense_block_1(x)
        x = self.transition_1(x)
        x = self.dense_block_2(x)
        x = self.transition_2(x)
        x = self.dense_block_3(x)

        x = self.avgpool(x)
        
        ## Processing age and gender inputs through their respective dense layers               
        age_features = self.age_dense(age_input)
        gender_features = self.gender_dense(gender_input)
        
        # Concatenate image features with age and gender features
        x = tf.concat([x, age_features, gender_features], axis=-1)
        
        x = self.fc(x)

        return x

In [9]:
def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [10]:
class ParameterTraining(object):
    def __init__(self, num_init_features, growth_rate, block_layers, compression_rate, 
                 drop_rate, no_epochs_1, no_epochs_2, lr_1, lr_2, n_bootstrap=1000):
        self.num_init_features = num_init_features
        self.growth_rate = growth_rate
        self.block_layers = block_layers
        self.compression_rate = compression_rate
        self.drop_rate = drop_rate
        self.no_epochs_1 = no_epochs_1
        self.no_epochs_2 = no_epochs_2
        self.lr_1 = lr_1
        self.lr_2 = lr_2
        self.n_bootstrap = n_bootstrap

    def _calculate_metrics(self, y_true, y_pred):
        """Calculate metrics with robust error handling"""
        try:
            # Handle input formats
            if len(y_pred.shape) == 2 and y_pred.shape[1] > 1:
                y_pred_class = np.argmax(y_pred, axis=1)
                y_pred_proba = y_pred[:, 1] if y_pred.shape[1] == 2 else y_pred
            else:
                y_pred_class = y_pred
                y_pred_proba = y_pred

            if len(y_true.shape) == 2 and y_true.shape[1] > 1:
                y_true_class = np.argmax(y_true, axis=1)
            else:
                y_true_class = y_true

            # Calculate metrics
            precision = metrics.precision_score(y_true_class, y_pred_class, zero_division=0)
            recall = metrics.recall_score(y_true_class, y_pred_class, zero_division=0)
            mcc = metrics.matthews_corrcoef(y_true_class, y_pred_class)
            cm = metrics.confusion_matrix(y_true_class, y_pred_class)
            
            # Handle specificity calculation
            specificity = 0.0
            if cm.shape == (2, 2):
                specificity = cm[0,0]/(cm[0,0]+cm[0,1]) if (cm[0,0]+cm[0,1]) > 0 else 0
            
            accuracy = metrics.accuracy_score(y_true_class, y_pred_class)
            
            # Handle AUC calculation
            auc = 0.5  # Default value for degenerate cases
            if len(np.unique(y_true_class)) >= 2:
                try:
                    auc = metrics.roc_auc_score(y_true_class, y_pred_proba)
                except:
                    pass

            return {
                'accuracy': accuracy,
                'precision': precision,
                'recall': recall,
                'specificity': specificity,
                'mcc': mcc,
                'auc': auc
            }
        except Exception as e:
            print(f"Error calculating metrics: {e}")
            return {
                'accuracy': np.nan,
                'precision': np.nan,
                'recall': np.nan,
                'specificity': np.nan,
                'mcc': np.nan,
                'auc': np.nan
            }

    def _bootstrap_ci(self, y_true, y_pred, n_bootstrap=1000):
        """Calculate bootstrap CIs with robust sampling"""
        boot_stats = {
            'accuracy': [],
            'precision': [],
            'recall': [],
            'specificity': [],
            'mcc': [],
            'auc': []
        }
        
        for _ in range(n_bootstrap):
            try:
                # Resample with replacement
                indices = resample(np.arange(len(y_true)))
                y_true_sample = y_true[indices]
                y_pred_sample = y_pred[indices]
                
                # Skip if only one class in sample
                if len(np.unique(np.argmax(y_true_sample, axis=1))) < 2:
                    continue
                
                metrics_dict = self._calculate_metrics(y_true_sample, y_pred_sample)
                
                # Only append valid metrics
                for key in boot_stats:
                    if not np.isnan(metrics_dict[key]):
                        boot_stats[key].append(metrics_dict[key])
            except Exception as e:
                print(f"Bootstrap sampling failed: {e}")
                continue
        
        # Calculate CIs only if we have enough samples
        ci_dict = {}
        for key in boot_stats:
            if len(boot_stats[key]) >= 30:  # Minimum samples for reliable CI
                ci_dict[f'{key}_ci'] = (
                    np.percentile(boot_stats[key], 2.5),
                    np.percentile(boot_stats[key], 97.5)
                )
            else:
                print(f"Warning: Not enough valid samples for {key} CI (n={len(boot_stats[key])})")
                ci_dict[f'{key}_ci'] = (np.nan, np.nan)
        
        return ci_dict

    def train(self, x_train, y_train, age_train, sex_train, kfold, seed, EarlyStop):
        """Main training method with cross-validation"""
        # Initialize metric containers
        cv_metrics = {
            'accuracy': [],
            'precision': [],
            'recall': [],
            'specificity': [],
            'mcc': [],
            'auc': []
        }
        
        # Store all predictions for final bootstrap
        all_val_preds = []
        all_val_true = []
        
        fold_no = 1
        for train_idx, val_idx in kfold.split(x_train, y_train):
            print(f"\n--- Processing Fold {fold_no}/{kfold.n_splits} ---")
            
            # Model initialization
            model = DenseNet(
                num_init_features=self.num_init_features[0],
                growth_rate=self.growth_rate[0],
                block_layers=self.block_layers,
                compression_rate=self.compression_rate[0],
                drop_rate=self.drop_rate[0]
            )
            
            # First training phase
            model.compile(
                loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(learning_rate=self.lr_1),
                metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
            )
            
            history = model.fit(
                [x_train[train_idx], age_train[train_idx], sex_train[train_idx]], 
                y_train[train_idx],
                batch_size=256,
                epochs=self.no_epochs_1,
                validation_data=([x_train[val_idx], age_train[val_idx], sex_train[val_idx]], y_train[val_idx]),
                verbose=0,
                callbacks=[EarlyStop]
            )
            
            # Second training phase
            model.compile(
                loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(learning_rate=self.lr_2),
                metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
            )
            
            history = model.fit(
                [x_train[train_idx], age_train[train_idx], sex_train[train_idx]], 
                y_train[train_idx],
                batch_size=256,
                epochs=self.no_epochs_2,
                validation_data=([x_train[val_idx], age_train[val_idx], sex_train[val_idx]], y_train[val_idx]),
                verbose=0,
                callbacks=[EarlyStop]
            )
            
            # Validation predictions
            y_pred_val = model.predict([x_train[val_idx], age_train[val_idx], sex_train[val_idx]])
            y_val = y_train[val_idx]
            
            # Store predictions for final bootstrap
            all_val_preds.append(y_pred_val)
            all_val_true.append(y_val)
            
            # Calculate fold metrics
            y_pred_class = np.argmax(y_pred_val, axis=1)
            y_true_class = np.argmax(y_val, axis=1)
            metrics_dict = self._calculate_metrics(y_true_class, y_pred_class)
            
            for key in cv_metrics:
                cv_metrics[key].append(metrics_dict[key])
            
            fold_no += 1
        
        # Concatenate all validation results
        all_val_preds = np.concatenate(all_val_preds)
        all_val_true = np.concatenate(all_val_true)
        all_val_true_class = np.argmax(all_val_true, axis=1)
        
        # Calculate cross-validation stats
        cv_stats = {}
        for key in cv_metrics:
            cv_stats[f'{key}_mean'] = np.nanmean(cv_metrics[key])
            cv_stats[f'{key}_std'] = np.nanstd(cv_metrics[key])
        
        # Calculate bootstrap CIs
        print("\nCalculating bootstrap confidence intervals...")
        bootstrap_cis = self._bootstrap_ci(all_val_true, all_val_preds, self.n_bootstrap)
        
        # Create results DataFrame
        results = {
            'Metric': ['Accuracy', 'Precision', 'Recall (Sensitivity)', 'Specificity', 'MCC', 'AUC'],
            'Mean': [
                cv_stats['accuracy_mean'],
                cv_stats['precision_mean'],
                cv_stats['recall_mean'],
                cv_stats['specificity_mean'],
                cv_stats['mcc_mean'],
                cv_stats['auc_mean']
            ],
            'Std': [
                cv_stats['accuracy_std'],
                cv_stats['precision_std'],
                cv_stats['recall_std'],
                cv_stats['specificity_std'],
                cv_stats['mcc_std'],
                cv_stats['auc_std']
            ],
            '95% CI': [
                f"({bootstrap_cis['accuracy_ci'][0]:.3f}, {bootstrap_cis['accuracy_ci'][1]:.3f})",
                f"({bootstrap_cis['precision_ci'][0]:.3f}, {bootstrap_cis['precision_ci'][1]:.3f})",
                f"({bootstrap_cis['recall_ci'][0]:.3f}, {bootstrap_cis['recall_ci'][1]:.3f})",
                f"({bootstrap_cis['specificity_ci'][0]:.3f}, {bootstrap_cis['specificity_ci'][1]:.3f})",
                f"({bootstrap_cis['mcc_ci'][0]:.3f}, {bootstrap_cis['mcc_ci'][1]:.3f})",
                f"({bootstrap_cis['auc_ci'][0]:.3f}, {bootstrap_cis['auc_ci'][1]:.3f})"
            ]
        }
        
        results_df = pd.DataFrame(results)
        
        # Format output
        def format_value(x):
            if isinstance(x, str):
                return x
            return f"{x:.1%}" if x <= 1 else f"{x:.3f}"
        
        for col in ['Mean', 'Std']:
            results_df[col] = results_df[col].apply(format_value)
        
        # Save results
        dir = os.path.join("/data/home/lmx/psy_test/v2_20250425/", "github")
        os.makedirs(seed_dir, exist_ok=True)
        model.save(dir)
                
        return {
            'accuracy': cv_stats['accuracy_mean'],
            'precision': cv_stats['precision_mean'],
            'recall': cv_stats['recall_mean'],
            'specificity': cv_stats['specificity_mean'],
            'mcc': cv_stats['mcc_mean'],
            'auc': cv_stats['auc_mean'],
            'results_df': results_df
        }

In [11]:
# ============ Main Execution ============ 
if __name__ == "__main__":
    # Define parameters
    num_folds = 10
    batch_size = 256
    no_epochs = 200

    num_init_features = [32]
    growth_rate = [8]
    block_layers = [3, 4, 3]
    compression_rate = [0.5]
    drop_rate = [0.3]
    no_epochs_1 = 10
    no_epochs_2 = 200
    lr_1 = 1e-3
    lr_2 = 1e-4

    EarlyStop = keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=20,
        min_delta=0,
        mode='auto',
        restore_best_weights=True
    )

    # Initialize training class
    PT = ParameterTraining(
        num_init_features=num_init_features,
        growth_rate=growth_rate,
        block_layers=block_layers,
        compression_rate=compression_rate,
        drop_rate=drop_rate,
        no_epochs_1=no_epochs_1,
        no_epochs_2=no_epochs_2,
        lr_1=lr_1,
        lr_2=lr_2,
        n_bootstrap=1000  # Number of bootstrap samples
    )

    # Shuffle data (assuming x_train, y_train, etc. are defined)
    shuffle_idx = np.random.permutation(np.arange(len(x_train)))
    x_train = x_train[shuffle_idx]
    y_train = y_train[shuffle_idx]
    age_train = age_train[shuffle_idx]
    sex_train = sex_train[shuffle_idx]

    # Initialize KFold
    kfold = KFold(n_splits=num_folds, shuffle=True)

    # Run training
    results = PT.train(
        x_train=x_train,
        y_train=y_train,
        age_train=age_train,
        sex_train=sex_train,
        kfold=kfold,
        seed=seed,
        EarlyStop=EarlyStop
    )


print("\n=== Training Completed ===")


--- Processing Fold 1/10 ---


2025-05-07 10:45:09.811353: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2025-05-07 10:45:12.389229: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2025-05-07 10:45:12.619321: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7



--- Processing Fold 2/10 ---

--- Processing Fold 3/10 ---

--- Processing Fold 4/10 ---


KeyboardInterrupt: 